# DESCRIPTION AT PATIENT LEVEL

## Imports

In [334]:
import pandas as pd
import pip
#pip.main(["install", "openpyxl"])
from IPython.display import display
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [335]:
os.getcwd()

'/Users/medscoops/neuefische/Respiratory_Disease_Classification/Client_side_app/web'

## Data Management

### Kaggle1 - Diagnosis

In [336]:
kaggle1_diagnosis = pd.read_csv('ICBHI_diagnosis.txt', sep='\t', header=None, names=['id', 'diagnosis'])
print(kaggle1_diagnosis.diagnosis.unique())
kaggle1_diagnosis.head()


['URTI' 'Healthy' 'Asthma' 'COPD' 'LRTI' 'Bronchiectasis' 'Pneumonia'
 'Bronchiolitis']


,id,diagnosis
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


### Kaggle1 - Age and Gender

In [338]:
kaggle1_age_gender = pd.read_csv('demographic_info.txt', sep='\t', header=None, names=['id', 'diagnosis'])
# assert that this second diagnosis column is empty
print(kaggle1_age_gender.diagnosis.unique())
# extract new-id, age, and gender from the id column
print(kaggle1_age_gender.id.str.split(' ', 3).head())
kaggle1_age_gender[['new_id', 'age', 'gender', 'rest']] = kaggle1_age_gender.id.str.split(' ', 3, expand=True)
kaggle1_age_gender.drop(['id', 'diagnosis', 'rest'], axis=1, inplace=True)
kaggle1_age_gender.head()

[nan]
0        [101, 3, F, NA 19 99]
1    [102, 0.75, F, NA 9.8 73]
2       [103, 70, F, 33 NA NA]
3    [104, 70, F, 28.47 NA NA]
4       [105, 7, F, NA 32 135]
Name: id, dtype: object


/var/folders/vp/9cqxp4sx7z7bnsq36dn0zx240000gn/T/ipykernel_31548/1823203254.py:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  print(kaggle1_age_gender.id.str.split(' ', 3).head())
/var/folders/vp/9cqxp4sx7z7bnsq36dn0zx240000gn/T/ipykernel_31548/1823203254.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  kaggle1_age_gender[['new_id', 'age', 'gender', 'rest']] = kaggle1_age_gender.id.str.split(' ', 3, expand=True)


,new_id,age,gender
0,101,3,F
1,102,0.75,F
2,103,70,F
3,104,70,F
4,105,7,F


### Full Kaggle1

In [340]:
kaggle1 = pd.concat([kaggle1_diagnosis, kaggle1_age_gender], axis=1)
#assert that id and new_id are identical (correct sorting)
print(kaggle1.info())
print(kaggle1.id.equals(kaggle1.new_id.astype('int64'))) #first lines for more security: print (kaggle1.id == kaggle1.new_id.astype('int64').head())
kaggle1.drop(['new_id'], axis=1, inplace=True)
#distinguish Kaggle1 ids from other ids
kaggle1.id = 'k1_' + kaggle1.id.astype(str)
#remove strings from age
kaggle1.age = kaggle1.age.str.replace('NA', 'NAN')
kaggle1.age = kaggle1.age.astype(float)
kaggle1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         126 non-null    int64 
 1   diagnosis  126 non-null    object
 2   new_id     126 non-null    object
 3   age        126 non-null    object
 4   gender     126 non-null    object
dtypes: int64(1), object(4)
memory usage: 5.0+ KB
None
True


,id,diagnosis,age,gender
0,k1_101,URTI,3.00,F
1,k1_102,Healthy,0.75,F
2,k1_103,Asthma,70.00,F
3,k1_104,COPD,70.00,F
4,k1_105,URTI,7.00,F


### Mendeley1

In [342]:
path = 'Data annotation.xlsx'
print(pd.read_excel(path).head())
#check if the xlsx file has empty lines
print(pd.read_excel(path).tail())

    Age Gender Location Sound type                       Diagnosis  \
0  70.0      M    P L L      I E W                          Asthma   
1  52.0      F   P L L         E W                          Asthma   
2  50.0      F   P L L       I E W                          Asthma   
3  72.0      F   P R L         I C  Heart Failure + Lung Fibrosis    
4  71.0      M   P R L     I C E W            Heart Failure + COPD   

   Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  \
0         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN   

  Posterior: P Lower: L Left: L Right R, UPPER: U, ANTERIOR: A, MIDDLE: M  
0  Insipratory: I, Experiatory: E, Wheezes: W, Cr...                       
1                                  Crep:crepitations                       
2                               

In [343]:
#take the columns of interest (same as in Kaggle1)
mendeley1 = pd.concat([pd.read_excel(path).Diagnosis, pd.read_excel(path).Age, pd.read_excel(path).Gender], axis=1)
#make an id column, adapt the names (same as in Kaggle1), drop empty lines, and distinguish Mendeley1 from other ids
mendeley1 = mendeley1.reset_index()
mendeley1 = mendeley1.rename({'index':'id', 'Diagnosis':'diagnosis', 'Age':'age', 'Gender':'gender'}, axis=1).dropna()
mendeley1.id = 'm1_' + mendeley1.id.astype(str)
#check if the Mendeley1 still has empty lines
print(mendeley1.tail())
mendeley1.head()

         id diagnosis   age gender
107  m1_107      COPD  63.0      M
108  m1_108         N  26.0      M
109  m1_109      COPD  62.0      M
110  m1_110      COPD  51.0      M
111  m1_111         N  30.0      M


,id,diagnosis,age,gender
0,m1_0,Asthma,70.0,M
1,m1_1,Asthma,52.0,F
2,m1_2,Asthma,50.0,F
3,m1_3,Heart Failure + Lung Fibrosis,72.0,F
4,m1_4,Heart Failure + COPD,71.0,M


### Full dataset

In [345]:
df = pd.concat([kaggle1, mendeley1], axis=0)
# search for categories
print(df.info())
print(df.diagnosis.unique())
print(df.gender.unique())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 111
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         238 non-null    object 
 1   diagnosis  238 non-null    object 
 2   age        237 non-null    float64
 3   gender     238 non-null    object 
dtypes: float64(1), object(3)
memory usage: 9.3+ KB
None
['URTI' 'Healthy' 'Asthma' 'COPD' 'LRTI' 'Bronchiectasis' 'Pneumonia'
 'Bronchiolitis' 'Heart Failure + Lung Fibrosis ' 'Heart Failure + COPD'
 'Plueral Effusion' 'Heart Failure' 'heart failure' 'pneumonia'
 'Asthma and lung fibrosis' 'BRON' 'Lung Fibrosis' 'copd' 'asthma' 'N']
['F' 'M' 'NA']


,id,diagnosis,age,gender
0,k1_101,URTI,3.00,F
1,k1_102,Healthy,0.75,F
2,k1_103,Asthma,70.00,F
3,k1_104,COPD,70.00,F
4,k1_105,URTI,7.00,F


### Correct classification

healthy: 'Healthy', 'N'

chronic:
- obstructive:
    - chronic_obstructive_pulmonary_disease: 'COPD', 'copd', 'Heart Failure + COPD'
    - bronchial_asthma: 'Asthma', 'asthma', 'Asthma and lung fibrosis'
- restrictive_lung_disease: 'Lung Fibrosis', 'Heart Failure + Lung Fibrosis '
- destructive_lung_disease: 'Bronchiectasis', 'BRON'

acute:
- infectious:
    - upper_airway_infection (rhinitis, laryngitis): 'URTI'
    - lower_airway_infection (trachea, bronchi, bronchioles, alveoli): 'LRTI', 'Bronchiolitis', 'Pneumonia', 'pneumonia'
- exacerbation:
    - pleural_effusion: 'Plueral Effusion'

heart_failure_unknown_origin: 'Heart Failure', 'heart failure'

In [346]:
labels_map = {
    'Healthy':'healthy', 'N':'healthy',
    'COPD':'chronic_obstructive_pulmonary_disease','copd':'chronic_obstructive_pulmonary_disease','Heart Failure + COPD':'chronic_obstructive_pulmonary_disease',
    'Asthma':'bronchial_asthma','asthma':'bronchial_asthma','Asthma and lung fibrosis':'bronchial_asthma',
    'Lung Fibrosis':'restrictive_lung_disease', 'Heart Failure + Lung Fibrosis ':'restrictive_lung_disease',
    'Bronchiectasis':'destructive_lung_disease', 'BRON':'destructive_lung_disease',
    'URTI':'upper_airway_infection',
    'LRTI':'lower_airway_infection', 'Bronchiolitis':'lower_airway_infection', 'Pneumonia':'lower_airway_infection', 'pneumonia':'lower_airway_infection',
    'Plueral Effusion':'pleural_effusion',
    'Heart Failure':'heart_failure_unknown_origin', 'heart failure':'heart_failure_unknown_origin',
}
df["diagnosis"] = df[
    "diagnosis"
].replace(labels_map)
print(df.diagnosis.unique())

['upper_airway_infection' 'healthy' 'bronchial_asthma'
 'chronic_obstructive_pulmonary_disease' 'lower_airway_infection'
 'destructive_lung_disease' 'restrictive_lung_disease' 'pleural_effusion'
 'heart_failure_unknown_origin']


### Column distinguishing between chronic and acute

In [347]:
df['chronic'] = np.where(
    df['diagnosis'].isin(['bronchial_asthma','chronic_obstructive_pulmonary_disease',
    'restrictive_lung_disease','destructive_lung_disease']), 'yes', 'no')
df.loc[df['diagnosis'] == 'healthy', 'chronic'] = 'healthy'
# remember that one for more complex situations:
    #conditions = [df.A.eq(df.B), df.A.gt(df.B), df.A.lt(df.B)]
    #choices = [0, 1, -1]
    #df['C'] = np.select(conditions, choices)

### Column distinguishing between healthy and diseased

In [348]:
df['diseased'] = np.where(df['chronic']!='healthy','yes','no')

### Columns for cycle duration

In [349]:
#prepare the columns
df[['dur_mean','dur_sd','dur_n']] = None

In [350]:
df.head()

,id,diagnosis,age,gender,chronic,diseased,dur_mean,dur_sd,dur_n
0,k1_101,upper_airway_infection,3.00,F,no,yes,None,None,None
1,k1_102,healthy,0.75,F,healthy,no,None,None,None
2,k1_103,bronchial_asthma,70.00,F,yes,yes,None,None,None
3,k1_104,chronic_obstructive_pulmonary_disease,70.00,F,yes,yes,None,None,None
4,k1_105,upper_airway_infection,7.00,F,no,yes,None,None,None


In [353]:
#check a Kaggle1 file
pd.read_csv('txt_files/101_1b1_Al_sc_Meditron.txt', sep='\t', header=None).head()

,0,1,2,3
0,0.036,0.579,0,0
1,0.579,2.450,0,0
2,2.450,3.893,0,0
3,3.893,5.793,0,0
4,5.793,7.521,0,0


In [352]:
#fill the columns for the Kaggle1 dataset
path = 'txt_files/'
for line in range(df.shape[0]):
    #if the participant is on Kaggle1
    if df.iloc[line, 0].split('_')[0]=='k1':
        temp = pd.DataFrame(columns=['start', 'end', 'rest1', 'rest2'])
        for file in os.listdir(path):
            #Kaggle1 filenames that start with the second part of the id correspond to the id
            if file.startswith(df.iloc[line, 0].split('_')[1]):
                #concatenate the txt file to temp
                temp = pd.concat([temp,pd.read_csv(os.path.join(path, file), sep='\t', header=None, names=['start', 'end', 'rest1', 'rest2'])], axis=0)
        df.iloc[line, 6] = np.mean(temp.end - temp.start)
        df.iloc[line, 7] = np.std(temp.end - temp.start)
        df.iloc[line, 8] = temp.shape[0]

In [354]:
#check a temp file
temp

,start,end,rest1,rest2
0,0.022,0.979,0,0
1,0.979,2.864,0,0
2,2.864,4.921,1,0
3,4.921,7.179,1,0
4,7.179,9.336,0,0
5,9.336,11.393,0,0
6,11.393,13.536,0,0
7,13.536,15.907,0,0
8,15.907,18.307,0,0
9,18.307,19.964,0,0


In [358]:
#destring the columns
df['dur_mean'] = df.dur_mean.astype(float)
df['dur_sd'] = df.dur_sd.astype(float)
df['dur_n'] = df.dur_n.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 111
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         238 non-null    object 
 1   diagnosis  238 non-null    object 
 2   age        237 non-null    float64
 3   gender     238 non-null    object 
 4   chronic    238 non-null    object 
 5   diseased   238 non-null    object 
 6   dur_mean   126 non-null    float64
 7   dur_sd     126 non-null    float64
 8   dur_n      126 non-null    float64
dtypes: float64(4), object(5)
memory usage: 18.6+ KB


### Finite dataset

In [359]:
df.head()

,id,diagnosis,age,gender,chronic,diseased,dur_mean,dur_sd,dur_n
0,k1_101,upper_airway_infection,3.00,F,no,yes,1.731652,0.398486,23.0
1,k1_102,healthy,0.75,F,healthy,no,1.515385,0.387094,13.0
2,k1_103,bronchial_asthma,70.00,F,yes,yes,3.269167,0.602692,6.0
3,k1_104,chronic_obstructive_pulmonary_disease,70.00,F,yes,yes,2.132018,1.041390,55.0
4,k1_105,upper_airway_infection,7.00,F,no,yes,2.483875,0.636807,8.0


In [360]:
df.tail()

,id,diagnosis,age,gender,chronic,diseased,dur_mean,dur_sd,dur_n
107,m1_107,chronic_obstructive_pulmonary_disease,63.0,M,yes,yes,NaN,NaN,NaN
108,m1_108,healthy,26.0,M,healthy,no,NaN,NaN,NaN
109,m1_109,chronic_obstructive_pulmonary_disease,62.0,M,yes,yes,NaN,NaN,NaN
110,m1_110,chronic_obstructive_pulmonary_disease,51.0,M,yes,yes,NaN,NaN,NaN
111,m1_111,healthy,30.0,M,healthy,no,NaN,NaN,NaN


## Data Description

In [361]:
# Full dataset
print(df.gender.value_counts()[1]/237)
print(df.age.describe())
df.dur_mean.describe()

0.3755274261603376
count    237.000000
mean      46.519325
std       27.075867
min        0.250000
25%       21.000000
50%       56.000000
75%       70.000000
max       93.000000
Name: age, dtype: float64


count    126.000000
mean       2.697041
std        0.960348
min        0.826208
25%        2.067220
50%        2.489312
75%        3.292731
max        4.987375
Name: dur_mean, dtype: float64

In [362]:
#By health status
print(df.diseased.value_counts())
print(pd.concat([df, pd.get_dummies(df.gender)], axis=1).groupby(['diseased']).F.sum() / df.groupby(['diseased']).gender.count())
print(df.groupby(['diseased']).age.describe())
df.groupby(['diseased']).dur_mean.describe()
#Might useful:
    #print(df.groupby(['diseased']).gender.value_counts().unstack(fill_value=0))
    #labels, uniques = pd.factorize(df.gender)

yes    177
no      61
Name: diseased, dtype: int64
diseased
no     0.393443
yes    0.367232
dtype: float64
          count       mean        std   min    25%   50%   75%   max
diseased                                                            
no         61.0  27.379672  24.081397  0.25   5.00  24.0  38.0  81.0
yes       176.0  53.152955  24.856916  0.58  42.75  60.0  71.0  93.0


,count,mean,std,min,25%,50%,75%,max
diseased,,,,,,,,
no,26.0,2.357992,0.957300,1.106389,1.811955,2.211611,2.834536,4.985500
yes,100.0,2.785193,0.946055,0.826208,2.123690,2.698546,3.344020,4.987375


In [363]:
#By chronicity
print(df.chronic.value_counts())
print(pd.concat([df, pd.get_dummies(df.gender)], axis=1).groupby(['chronic']).F.sum() / df.groupby(['chronic']).gender.count())
print(df.groupby(['chronic']).age.describe())
df.groupby(['chronic']).dur_mean.describe()

yes        124
healthy     61
no          53
Name: chronic, dtype: int64
chronic
healthy    0.393443
no         0.415094
yes        0.346774
dtype: float64
         count       mean        std    min   25%   50%   75%   max
chronic                                                            
healthy   61.0  27.379672  24.081397   0.25   5.0  24.0  38.0  81.0
no        53.0  36.432453  32.034261   0.58   3.0  36.0  70.0  83.0
yes      123.0  60.357724  16.586154  12.00  52.5  63.0  71.5  93.0


,count,mean,std,min,25%,50%,75%,max
chronic,,,,,,,,
healthy,26.0,2.357992,0.957300,1.106389,1.811955,2.211611,2.834536,4.985500
no,28.0,2.216848,1.014573,0.826208,1.681989,2.027698,2.484323,4.985750
yes,72.0,3.006217,0.824387,1.404730,2.257828,2.847929,3.522973,4.987375


In [364]:
#count diagnoses
df.diagnosis.value_counts()

chronic_obstructive_pulmonary_disease    75
healthy                                  61
bronchial_asthma                         34
lower_airway_infection                   19
heart_failure_unknown_origin             18
upper_airway_infection                   14
destructive_lung_disease                 10
restrictive_lung_disease                  5
pleural_effusion                          2
Name: diagnosis, dtype: int64

# More

In [257]:
#alternative way of walking trough files
rootdir = 'txt_files/'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(os.path.join(file).split('_')[0]) #.join(subdir, file)

148
130
107
160
174
114
138
130
141
193
162
198
156
115
174
224
158
147
151
107
216
163
177
155
193
173
170
130
203
130
138
218
160
166
176
211
104
130
203
188
172
149
205
178
179
158
160
186
220
178
113
176
122
154
117
218
172
193
200
130
185
156
174
160
164
112
201
157
134
162
122
133
145
158
196
158
133
186
130
172
198
130
207
221
203
170
202
139
203
178
200
223
109
138
130
205
151
101
107
129
130
175
172
147
107
213
147
151
186
195
145
211
130
158
178
154
162
130
221
191
130
198
177
154
130
158
113
151
111
177
.DS
172
156
185
218
204
193
177
205
175
186
122
217
138
182
160
195
176
147
208
158
108
213
165
120
203
188
170
134
151
205
201
107
147
118
213
207
142
110
178
174
200
186
133
109
223
176
147
213
178
207
192
186
133
162
198
138
154
200
163
225
193
170
107
200
174
130
133
159
138
107
207
124
170
172
169
191
130
180
104
112
160
138
162
146
163
144
158
127
203
128
151
135
170
141
213
206
176
110
162
176
213
207
151
122
156
186
133
162
198
138
154
174
200
163
204
193
221
172
170


In [269]:
# example with exec
path = 'txt_files/'
a = 0
for file in os.listdir(path):
    if file.startswith('101'):
        a += 1
        print(os.path.join(path, file))
        x = 'test'+str(a)+" = pd.read_csv(os.path.join(path, file), sep='\t', header=None, names=['start', 'end', 'rest1', 'rest2'])"
        exec(x)

txt_files/101_1b1_Al_sc_Meditron.txt
txt_files/101_1b1_Pr_sc_Meditron.txt
